In [34]:
import os
from dotenv import load_dotenv
load_dotenv() #get environment variables from .env file

True

### Instiantiate TwelveLabs Client with API KEY

In [80]:
from twelvelabs import TwelveLabs

client = TwelveLabs(api_key=os.getenv("API_KEY"))

Create an Index  *(ONLY RUN ONCE)*

In [37]:
from twelvelabs import APIStatusError

index_object = None

try:
    print("Creating Index... \n")
    index_object = client.index.create(
        name = "IndexfromLAPTOP",
        engines = [{"name" : "marengo2.6", "options" : ["visual"]},{"name" : "pegasus1", "options" : ["visual"]}]
    )
    print(index_object)
except APIStatusError as e:
    print("API Status Error, 4xx or 5xx : ",e)
except Exception as e:
    print(e)


Creating Index... 

Index(id='66844735eb3979a66bdc9f94', created_at='2024-07-02T18:30:13.506Z', updated_at='2024-07-02T18:30:13.506Z', name='IndexfromLAPTOP', engines=RootModelList[Engine](root=[Engine(name='pegasus1', options=['visual'], addons=None), Engine(name='marengo2.6', options=['visual'], addons=None)]), video_count=0, total_duration=0.0, expires_at='2024-09-30T18:30:13.506Z')


In [81]:
indexes = client.index.list().root
for index in indexes:
    index_id = index.id
    index_name = index.name
    index_engines = index.engines.root
    
    print("Index ID:", index_id)
    print("Index Name:", index_name)
    print("Index Engines:", index_engines)
    print("---")

Index ID: 66844735eb3979a66bdc9f94
Index Name: IndexfromLAPTOP
Index Engines: [Engine(name='pegasus1', options=['visual'], addons=None), Engine(name='marengo2.6', options=['visual'], addons=None)]
---
Index ID: 66841d79eb3979a66bdc9f30
Index Name: Index1
Index Engines: [Engine(name='marengo2.6', options=['visual', 'conversation', 'text_in_video', 'logo'], addons=None), Engine(name='pegasus1', options=['visual', 'conversation'], addons=None)]
---
Index ID: 66841c757e3a05b242e8b122
Index Name: My Index (Default)
Index Engines: [Engine(name='marengo2.6', options=['visual', 'conversation', 'text_in_video', 'logo'], addons=None), Engine(name='pegasus1', options=['visual', 'conversation'], addons=None)]
---


In [154]:
videos = client.index.video.list("66844735eb3979a66bdc9f94").root
for vid in videos:
    video_id = vid.id
    video_name = vid.metadata.filename
    
    print("Video ID:", video_id)
    print("Video NAme:", video_name)

Video ID: 66844ca6d22b3a3c97bf39da
Video NAme: LBJ.mp4
Video ID: 66844ca4d22b3a3c97bf39d9
Video NAme: tml-draft.mov
Video ID: 66844c9fd22b3a3c97bf39d8
Video NAme: plays.mp4


Uploading Video to Index

In [87]:
from glob import glob
from twelvelabs.models.task import Task
from twelvelabs import BadRequestError

video_files = glob("./*.m[op][v4]")

for vid in video_files:
    print(f"Uploading {vid}")
    try:
        task = client.task.create(index_id="66844735eb3979a66bdc9f94", file=vid, language="en")
        print(f"Task id={task.id}\n")
    except BadRequestError as e:
        print(f"Error with task: {e}")
    
 #Optional) Monitor the video indexing process
# Utility function to print the status of a video indexing task
def on_task_update(task: Task):
        print(f"  Status={task.status}")
try:        
    task.wait_for_done(callback=on_task_update)
    if task.status != "ready":
        raise RuntimeError(f"Indexing failed with status {task.status}")
    print(f"Uploaded {vid}. The unique identifer of your video is {task.video_id}.")
except Exception as e:
    print(f"Error : {e}")

Uploading ./plays.mp4
Task id=66844c9feb3979a66bdc9fae

Uploading ./tml-draft.mov
Task id=66844ca3eb3979a66bdc9fb1

Uploading ./LBJ.mp4
Task id=66844ca5eb3979a66bdc9fb4

  Status=validating
  Status=validating
  Status=validating
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=pending
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Sta

In [165]:
#Search through video

result = client.search.query("66844735eb3979a66bdc9f94",
                             ["visual"],
                             query_text="Kobe Bryant",
                             group_by="clip",
                             threshold="high",sort_option="score")


In [166]:
import math

def convert_seconds(seconds):
    """Converts seconds to hours, minutes, and seconds in format HH:MM:SS or MM:SS"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds_remaining = math.floor(seconds % 60)
    
    if hours > 0:
        return f"{hours}:{minutes:02}:{seconds_remaining:02}"
    else:
        return f"{minutes}:{seconds_remaining:02}"
    
    
# from moviepy.video.io.VideoFileClip import VideoFileClip

# def clip_video(video_file, start_seconds, end_seconds, output_file='output.mp4'):
#     """This requires ffmpeg to be installed in computer"""
#     # Load video clip
#     video = VideoFileClip(video_file)

#     # Ensure end time does not exceed video duration
#     end_seconds = min(end_seconds, video.duration)

#     # Extract the subclip
#     subclip = video.subclip(start_seconds, end_seconds)

#     # Write the subclip to a file
#     subclip.write_videofile(output_file, codec='libx264', audio_codec='aac')

#     # Close the video file
#     video.reader.close()
#     video.audio.reader.close_proc()



In [167]:
for clip in result.data:
      print(
            f"video={video_name}  score={clip.score} start={convert_seconds(clip.start)} end={convert_seconds(clip.end)} confidence={clip.confidence}"
        )



video=plays.mp4  score=87.27 start=7:29 end=7:39 confidence=high
video=plays.mp4  score=87.03 start=6:33 end=6:37 confidence=high
video=plays.mp4  score=86.91 start=3:05 end=3:11 confidence=high
video=plays.mp4  score=86.41 start=0:18 end=0:21 confidence=high
video=plays.mp4  score=86.27 start=0:03 end=0:07 confidence=high
video=plays.mp4  score=86.21 start=7:11 end=7:22 confidence=high
video=plays.mp4  score=86.17 start=4:55 end=5:02 confidence=high
video=plays.mp4  score=86.15 start=3:31 end=3:48 confidence=high
video=plays.mp4  score=86.13 start=4:29 end=4:34 confidence=high
video=plays.mp4  score=86.13 start=5:04 end=5:09 confidence=high


In [168]:
while True:
    try:
        next_page_data = next(result)
        print(f"Next page's data \n")
        for clip in next_page_data:
            print(
                 f"video={video_name}  score={clip.score} start={convert_seconds(clip.start)} end={convert_seconds(clip.end)} confidence={clip.confidence}"
            )
    except StopIteration:
        print("\nThere is no next page in search result")
        break

Next page's data 

video=plays.mp4  score=86.07 start=6:24 end=6:33 confidence=high
video=plays.mp4  score=86.06 start=2:35 end=2:40 confidence=high
video=plays.mp4  score=85.89 start=3:11 end=3:13 confidence=high
video=plays.mp4  score=85.86 start=9:29 end=9:36 confidence=high
video=plays.mp4  score=85.76 start=5:20 end=5:32 confidence=high
video=plays.mp4  score=85.76 start=8:28 end=8:31 confidence=high
video=plays.mp4  score=85.74 start=5:51 end=6:06 confidence=high
video=plays.mp4  score=85.74 start=6:37 end=6:42 confidence=high
video=plays.mp4  score=85.63 start=2:08 end=2:17 confidence=high
video=plays.mp4  score=85.63 start=9:20 end=9:29 confidence=high
Next page's data 

video=plays.mp4  score=85.57 start=9:15 end=9:18 confidence=high
video=plays.mp4  score=85.53 start=1:33 end=1:48 confidence=high
video=plays.mp4  score=85.48 start=0:30 end=0:38 confidence=high
video=plays.mp4  score=85.48 start=4:01 end=4:29 confidence=high
video=plays.mp4  score=85.43 start=4:12 end=4:18 con